In [ ]:
from fastai.vision.all import *

In [ ]:
help(untar_data)

In [ ]:
from nbdev.showdoc import show_doc
show_doc(untar_data)

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
set_seed(42)

In [ ]:
path.ls()[:3]

In [ ]:
(path/'images').ls()[:3]

In [ ]:
path = untar_data(URLs.PETS)
fnames = get_image_files(path/'images')
pat = r'(.+)_\d+.jpg$'

In [ ]:
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=64

In [ ]:
dls = ImageDataLoaders.from_name_re(
    path,
    fnames,
    pat,
    item_tfms=item_tfms,
    batch_tfms=batch_tfms,
    bs=bs
)

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(),
                 get_y=RegexLabeller(pat = r'/([^/]+)_\d+.*'),
                 item_tfms=item_tfms,
                 batch_tfms=batch_tfms)

In [ ]:
path_im = path/'images'

In [ ]:
dls = pets.dataloaders(path_im, bs=bs)

In [ ]:
dls.show_batch(max_n=9, figsize=(6,7))

In [ ]:
dls.vocab

In [ ]:
dls.vocab.o2i

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage_1')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,10))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=3)

In [ ]:
learn.load('stage_1');

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(4, lr_max=slice(1e-6, 1e-4))

In [ ]:
learn.save('stage_2')